In [ ]:
__author__ = 'Vinicius Placco <vinicius.placco@noirlab.edu>'
__version__ = '20240129' # yyyymmdd; version datestamp of this notebook
#__datasets__ = ['allwise','smash']
__keywords__ = ['niri','gemini','supernova','dragons']

# Gemini NIRI reduction using DRAGONS Python API
***
## Public archival data from niriimg_tutorial - GN-2015B-Q-31 (SN2014J)
#### adapted from https://dragons.readthedocs.io/projects/niriimg-drtutorial/en/v3.1.0/index.html
#### don't forget to `conda install -n dragons nb_conda_kernels ipykernel` to run this notebook on the DRAGONS env 
***

## Table of contents
* [Goals](#goals)
* [Summary](#summary)
* [Disclaimers and attribution](#disclaimer)
* [Imports and setup](#imports)
* [About the dataset](#About)
* [Downloading data for reduction](#Downloading_Data)
* [Set up the DRAGONS logger](#DRAGONS_logger)
* [Create File Lists](#File_Lists)
* [Create Master Dark](#Master_dark)
* [Bad Pixel Mask](#BPM)
* [Create Master Flat Field](#Master_Flat)
* [Standard Star](#Standard_Star)
* [Reduce Science Images](#Reduce_Science)
* [Display stacked final image](#Display_Image)
* [Clean-up (optional)](#Clean-up)

<a class="anchor" id="goals"></a>
# Goals
Showcase how to perform NIRI imaging data reduction using the Gemini DRAGONS package on the Data Lab science platform. Uses a custom DRAGONS kernel `"DRAGONS (Py3.7)"`. The steps include downloading data from the Gemini archive, setting up a DRAGONS calibration service, processing of flats, darks, bad pixel mask, and science frames, and finally the creation of a single combined stacked image.

<a class="anchor" id="summary"></a>
# Summary
DRAGONS is a Python-based astronomical data reduction platform written by the Gemini Science User Support Department. It currently can be used to reduce imaging data from Gemini instruments GMOS, NIRI, Flamingos 2, GSAOI, and GNIRS, and spectroscopic data in GMOS longslit mode. Linked here is a general list of guides, manuals, and tutorials about the use of DRAGONS:
https://dragons.readthedocs.io/en/v3.1.0/

The DRAGONS kernel has been made available in the Data Lab environment, which should allow users to access the routines without being dependent on installing the software in their local machines. 

In this notebook, we present an example of a DRAGONS Jupyter notebook that works in the Data Lab environment to fully reduce example Gemini North NIRI H-band imaging data.
This is a version of the DRAGONS Jupyter notebook tutorial presented here: 
https://dragons.readthedocs.io/projects/niriimg-drtutorial/en/v3.1.0/ex1_niriim_extended_api.html

This notebook will not present all of the details of the many options available to adjust or optimize the DRAGONS NIRI data reduction process, rather will just show one example of a standard reduction of a GNIRS imaging dataset. 
More extensive explanations can be found in the general DRAGONS GNIRS data reduction tutorial from Gemini linked here:
https://dragons.readthedocs.io/projects/niriimg-drtutorial/en/v3.1.0/index.html

The data used in this notebook example is NIRI H-band imaging from the Gemini archive of the Supernova SN2014J from the Gemini North program "SN2014J at very late phases", PI: Marten van Kerkwijk, program ID GN-2015B-Q-31. More program information is given here: https://archive.gemini.edu/programinfo/GN-2015B-Q-31.

<a class="anchor" id="disclaimer"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

* DRAGONS publication: Labrie et al., "DRAGONS - Data Reduction for Astronomy from Gemini Observatory North and South", ASPC, 523, 321L, https://ui.adsabs.harvard.edu/abs/2019ASPC..523..321L/abstract

* DRAGONS open source software publication: https://zenodo.org/record/7776065#.ZDg5qOzMLUI


<a class="anchor" id="imports"></a>
# Importing Python Libraries** (you'll probably have to install the `wget` and `ipympl` libraries)

In [ ]:
from __future__ import print_function

import glob
import wget

from gempy.adlibrary import dataselect
from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce
from gempy.utils import logutils

from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib.colors import LogNorm
from matplotlib.colors import Normalize
from matplotlib.colors import PowerNorm

from collections import namedtuple

#%matplotlib widget
%matplotlib inline

<a class="anchor" id="About"></a>
# About the dataset

This is a NIRI imaging observation of an extended source, a galaxy showing as a dense field of stars. The observation sequence uses an offset to a nearby blank portion of the sky to monitor the sky levels since there are no area in the science observation that is not “contaminated” by the galaxy.

The calibrations we use for this example include:

Darks for the science and sky offset frames.
Flats, as a sequence of lamps-on and lamps-off exposures.
Short darks to use with the flats to create a bad pixel mask.
A set of standard star observations.

| Observation Type | File name(s) | Purpose and Exposure (seconds) |
| :--- | :--- | :---: |
| Science | N20160102S0270-274 | on-target |
| Science | N20160102S0275-279 | on-sky |
| Science Darks | N20160102S0423-432 | 20 sec, like science |
| Flats | N20160102S0373-382  | Lamp on |
| Flats | N20160102S0363-372 | Lamp off |
| Short Darks | N20160103S0463-472 |  |
| Standard Star | N20160102S0295-299 |  |

<a class="anchor" id="Downloading_Data"></a>
# Downloading the data (direct link to .tar file hosted at Gemini - 48 Megabytes)

In [ ]:
# uncomment the line and run the cell
#
#wget.download("http://www.gemini.edu/sciops/data/software/datapkgs/niriimg_tutorial_datapkg-v1.tar")

**Downloading the data** (individual files - direct link to the Gemini Archive - 221 Megabytes)

In [ ]:
## uncomment the lines and run the cell
#
## Science 
#
#wget.download("http://archive.gemini.edu/file/N20160102S0270.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0271.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0272.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0273.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0274.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0275.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0276.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0277.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0278.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0279.fits")
#
## Science darks
#
#wget.download("http://archive.gemini.edu/file/N20160102S0423.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0424.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0425.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0426.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0427.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0428.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0429.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0430.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0431.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0432.fits")
#
## Flats 
#
#wget.download("http://archive.gemini.edu/file/N20160102S0363.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0364.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0365.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0366.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0367.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0368.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0369.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0370.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0371.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0372.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0373.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0374.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0375.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0376.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0377.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0378.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0379.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0380.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0381.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0382.fits")
#
## Short darks
#
#wget.download("http://archive.gemini.edu/file/N20160103S0463.fits")
#wget.download("http://archive.gemini.edu/file/N20160103S0464.fits")
#wget.download("http://archive.gemini.edu/file/N20160103S0465.fits")
#wget.download("http://archive.gemini.edu/file/N20160103S0466.fits")
#wget.download("http://archive.gemini.edu/file/N20160103S0467.fits")
#wget.download("http://archive.gemini.edu/file/N20160103S0468.fits")
#wget.download("http://archive.gemini.edu/file/N20160103S0469.fits")
#wget.download("http://archive.gemini.edu/file/N20160103S0470.fits")
#wget.download("http://archive.gemini.edu/file/N20160103S0471.fits")
#wget.download("http://archive.gemini.edu/file/N20160103S0472.fits")
#
## Standard star
#
#wget.download("http://archive.gemini.edu/file/N20160102S0295.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0296.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0297.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0298.fits")
#wget.download("http://archive.gemini.edu/file/N20160102S0299.fits")

**Create and move data to raw/ directory** (uncomment first)

In [ ]:
!mkdir raw/
!mv N2016*.fits raw/

In [ ]:
# Check header of one raw science image
#
tmp = fits.open("raw/N20160102S0270.fits")
tmp[0].header

<a class="anchor" id="DRAGONS_logger"></a>
# Setting up the DRAGONS logger

DRAGONS comes with a local calibration manager that uses the same calibration association rules as the Gemini Observatory Archive. This allows reduce to make requests to a local light-weight database for matching processed calibrations when needed to reduce a dataset.

This simply tells the system where to put the calibration database, the database that will keep track of the processed calibrations we are going to send to it.

In [ ]:
logutils.config(file_name='niri_data_reduction.log')

**Create a list of all the FITS files in the directory**

In [ ]:
all_files = glob.glob('raw/N2016*[0-9].fits')
all_files.sort()
#all_files

<a class="anchor" id="File_Lists"></a>
# Create file lists

This data set contains science and calibration frames. For some programs, it could have different observed targets and different exposure times depending on how you like to organize your raw data.

The DRAGONS data reduction pipeline does not organize the data for you. You have to do it. DRAGONS provides tools to help you with that.

The first step is to create input file lists. The tool “dataselect” helps with that. It uses Astrodata tags and “descriptors” to select the files and send the filenames to a text file that can then be fed to “reduce”. (See the Astrodata User Manual for information about Astrodata.)

**Two lists for the darks**

We have two sets of darks; one set for the science frames, the 20-second darks, and another for making the BPM, the 1-second darks. We will create two lists.

If you did not know the exposure times for the darks, you could have use a combination of “dataselect” to select all the darks (tag DARK) and feed that list to “showd” to show descriptor values, in this case exposure_time. (See the descriptors page for a complete list.)

**A list for the flats**

The flats are a sequence of lamp-on and lamp-off exposures. We just send all of them to one list.

**A list for the standard star**

The standard stars at Gemini are normally taken as partner calibration.

**A list for the science observations**

The science frames are all the IMAGE non-FLAT frames that are also not the standard. Since flats are tagged FLAT and IMAGE, we need to exclude the FLAT tag.


You can see the observation_class of all the data using “showd”. Here we will print the object name too.

In [ ]:
darks1s = dataselect.select_data(
    all_files, ['DARK'], [],
    dataselect.expr_parser('exposure_time==1'))

darks20s = dataselect.select_data(
    all_files, ['DARK'], [],
    dataselect.expr_parser('exposure_time==20'))

flats = dataselect.select_data(all_files, ['FLAT'])

stdstar = dataselect.select_data(
    all_files, [], [],
    dataselect.expr_parser('object=="FS 17"'))

target = dataselect.select_data(
    all_files, ['IMAGE'], ['FLAT'],
    dataselect.expr_parser('object!="FS 17"'))

<a class="anchor" id="Master_dark"></a>
# Create Master Dark

We first create the master dark for the science target, then add it to the calibration database. The name of the output master dark, N20160102S0423_dark.fits, is written to the screen at the end of the process.

In [ ]:
reduce_darks = Reduce()
reduce_darks.files.extend(darks20s)
reduce_darks.runr()

<a class="anchor" id="BPM"></a>
# Bad Pixel Mask

The DRAGONS Gemini data reduction package, geminidr, comes with a static NIRI bad pixel mask (BPM) that gets automatically added to all the NIRI data as they gets processed. The user can also create a supplemental, fresher BPM from the flats and recent short darks. That new BPM is later fed to “reduce” as a user BPM to be combined with the static BPM. Using both the static and a fresh BPM from recent data lead to a better representation of the bad pixels. It is an optional but recommended step.

The flats and the short darks are the inputs.

The flats must be passed first to the input list to ensure that the recipe library associated with NIRI flats is selected. We will not use the default recipe but rather the special recipe from that library called makeProcessedBPM.

The BPM produced is named N20160102S0373_bpm.fits.

In [ ]:
reduce_bpm = Reduce()
reduce_bpm.files.extend(flats)
reduce_bpm.files.extend(darks1s)
reduce_bpm.recipename = 'makeProcessedBPM'
reduce_bpm.runr()

bpm = reduce_bpm.output_filenames[0]

<a class="anchor" id="Master_Flat"></a>
# Master Flat Field

A NIRI master flat is created from a series of lamp-on and lamp-off exposures. Each flavor is stacked, then the lamp-off stack is subtracted from the lamp-on stack.

In [ ]:
reduce_flats = Reduce()
reduce_flats.files.extend(flats)
reduce_flats.uparms = [('addDQ:user_bpm', bpm)]
reduce_flats.runr()

<a class="anchor" id="Standard_Star"></a>
# Standard Star

The standard star is reduced more or less the same way as the science target (next section) except that darks frames are not obtained for standard star observations. Therefore the dark correction needs to be turned off.

The processed flat field that we added earlier to the local calibration database will be fetched automatically. The user BPM (optional, but recommended) needs to be specified by the user.

In [ ]:
reduce_std = Reduce()
reduce_std.files.extend(stdstar)
reduce_std.uparms = [('addDQ:user_bpm', bpm)]
reduce_std.uparms.append(('darkCorrect:do_cal', 'skip'))
reduce_std.runr()

<a class="anchor" id="Reduce_Science"></a>
# Reduce Science Images

The science target is an extended source. We need to turn off the scaling of the sky because the target fills the field of view and does not represent a reasonable sky background. If scaling is not turned off in this particular case, it results in an over-subtraction of the sky frame.

The sky frame comes from off-target sky observations. We feed the pipeline all the on-target and off-target frames. The software will split the on-target and the off-target appropriately.

The master dark and the master flat will be retrieved automatically from the local calibration database. Again, the user BPM needs to be specified on the command line.

The output stack units are in electrons (header keyword BUNIT=electrons). The output stack is stored in a multi-extension FITS (MEF) file. The science signal is in the “SCI” extension, the variance is in the “VAR” extension, and the data quality plane (mask) is in the “DQ” extension.

In [ ]:
reduce_target = Reduce()
reduce_target.files.extend(target)
reduce_target.uparms = [('addDQ:user_bpm', bpm)]
reduce_target.uparms.append(('skyCorrect:scale_sky', False))
reduce_target.runr()

<a class="anchor" id="Display_Image"></a>
# Display the Stacked Image

In [ ]:
image_file = "N20160102S0271_image.fits"
hdu_list = fits.open(image_file)
hdu_list.info()

In [ ]:
image_data = fits.getdata(image_file, ext=1)
print(image_data.shape)

In [ ]:
plt.figure(figsize = (15,15))
plt.imshow(image_data,cmap='bone',norm=Normalize(vmin=1, vmax=50000))
plt.show()

<a class="anchor" id="Clean-up"></a>
# Optional: remove raw data (uncomment lines before running)

In [ ]:
#!rm -rf raw/